In [1]:
# import Libraries
import os
import langchain
import pinecone 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# ## Divide the docs into chunks
# ### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
# def chunk_data(docs,chunk_size=800,chunk_overlap=50):
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
#     doc=text_splitter.split_documents(docs)
#     return docs

In [4]:
# documents=chunk_data(docs=doc)
# len(documents)

In [5]:
from chunker import main_chunk_function
extracted_text = """-- Page 1 --
Edelweiss
GENERAL
INSURANCE
Well Baby Well Mother- Add On
Wordings
Air Ambulance Cover
In consideration of the payment of additional premium to Us, We will cover the expenses incurred on air
ambulance services in respect of an Insured Person which are offered by
healthcare or an air
ambulance service provider and which have been used during the Policy Period to transfer the Insured
Person to
Hospital or transfer from one hospital to another with adequate emergency facilities for the
provision of Emergency Care, provided that:
1_
Our maximum liability under this Benefit for any and all claims arising during the
Year will be restricted to the Sum insured as stated in the Policy Schedule;
ii_
The maximum distance of travel undertaken is 150 kms. In case of distance travelled is
more than 150 kms, proportionate amount of expenses upto 150 kms shall be payable
Example: If insured has travelled a distance of 300kms
an air ambulance we will pay
50% of the total cost or Sum Insured whichever is lower: (EligibilitylActual distance travelled
150kms/300kms
0.5)
iii.
It is for a life
threatening emergency health condition/s of the Insured Person which requires
immediate and rapid ambulance transportation from the place where the Insured Person is
situated at the time of requiring Emergency Care to a hospital provided that the
transportation is for Medically Necessary Treatment, is certified in writing by a Medical
Practitioner; and road ambulance services cannot be provided.
iv.
Such air ambulance providing the services, should be duly licensed to operate as such by a
competent government Authority.
This cover is limited to transportation from the area of emergency to a Hospital providing
emergency care which is not available at the place of origin and from one medical centre to
another.
We will not cover:
a.
Transfer of the Insured Person from one medical facility to another medical
facility of
similar capabilities which provides a similar level of care.
b_
transportation of the Insured Person from Hospital to the Insured Person' $
residence after helshe has been discharged from the Hospital
C.
transportation O air ambulance expenses incurred outside the geographical scope
of India.
d.
Attempt at suicide
Injuries resulting from participation in acts of war or insurrection
f.
commission of unlawful acts
g.
Incidents involving use of drugs unless prescribed by a medical practitioner
h
We have
accepted
a claim under Section II.A.1 in respect of the Insured Person for the
same Accident/Illness for which air ambulance services were availed.
We shall not be liable if Medically Necessary Treatment can be provided at the Hospital
where the Insured Person is situated at the time of requiring Emergency Care.
Claims towards Air Ambulance expenses will be payable by mode of Reimbursement only"""

chunks = main_chunk_function(extracted_text) 
chunks

['-- Page 1 --\nEdelweiss\nGENERAL\nINSURANCE\nWell Baby Well Mother- Add On\nWordings\nAir Ambulance Cover\nIn consideration of the payment of additional premium to Us, We will cover the expenses incurred on air\nambulance services in respect of an Insured Person which are offered by\nhealthcare or an air\nambulance service provider and which have been used during the Policy Period to transfer the Insured\nPerson to\nHospital or transfer from one hospital to another with adequate emergency facilities for the\nprovision of Emergency Care, provided that:\n1_\nOur maximum liability under this Benefit for any and all claims arising during the\nYear will be restricted to the Sum insured as stated in the Policy Schedule;\nii_\nThe maximum distance of travel undertaken is 150 kms.',
 'insured as stated in the Policy Schedule;\nii_\nThe maximum distance of travel undertaken is 150 kms. In case of distance travelled is\nmore than 150 kms, proportionate amount of expenses upto 150 kms shall be 

In [ ]:
from langchain.schema import Document


documents = [Document(page_content=text) for text in chunks]
documents

[Document(metadata={}, page_content='-- Page 1 --\nEdelweiss\nGENERAL\nINSURANCE\nWell Baby Well Mother- Add On\nWordings\nAir Ambulance Cover\nIn consideration of the payment of additional premium to Us, We will cover the expenses incurred on air\nambulance services in respect of an Insured Person which are offered by\nhealthcare or an air\nambulance service provider and which have been used during the Policy Period to transfer the Insured\nPerson to\nHospital or transfer from one hospital to another with adequate emergency facilities for the\nprovision of Emergency Care, provided that:\n1_\nOur maximum liability under this Benefit for any and all claims arising during the\nYear will be restricted to the Sum insured as stated in the Policy Schedule;\nii_\nThe maximum distance of travel undertaken is 150 kms.'),
 Document(metadata={}, page_content='insured as stated in the Policy Schedule;\nii_\nThe maximum distance of travel undertaken is 150 kms. In case of distance travelled is\nmor

In [7]:
# embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains.question_answering import load_qa_chain

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Extract the text from the Document objects
# texts = [doc.page_content for doc in documents]

# # Generate embeddings
# embeddings = embedding_model.embed_documents(texts)
# print(len(embeddings), len(embeddings[0]))

C:\Users\Dell\AppData\Local\Temp\ipykernel_32908\482718892.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\miniconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]

In [9]:
from pinecone import Pinecone, ServerlessSpec

# Create client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "hackrx6"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # For all-MiniLM-L6-v2
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to index
index = pc.Index(index_name)

In [10]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [11]:
# index=Pinecone.from_documents(doc,embeddings,index_name=index_name)
vector_store.add_documents(documents=documents, ids=uuids)

['ed5063da-9870-438a-8955-c6b2c8e1dca2',
 '4db371f5-e555-400e-acde-9bd1260677ae',
 '2be5bd64-e80f-4665-baf3-e4a0f5d968cf',
 'a1736466-8425-4da4-9fb8-c56094ba1031',
 'ca861f24-abbf-4382-a178-0295bd6b562c']

In [20]:
from langchain.chains import RetrievalQA  
from langchain_huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(
    # repo_id="deepseek-ai/DeepSeek-R1",
    # repo_id= "google/flan-t5-large",
    # task="text-generation", 
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    # task="text-generation",
    provider="novita", 
    # provider='together',
    huggingfacehub_api_token=os.getenv("HF_KEY")
)
model = ChatHuggingFace(llm=llm)
qa = RetrievalQA.from_chain_type(  
    llm=model,  
    chain_type="stuff",  
    retriever=vector_store.as_retriever()  
)  

In [21]:
qa.invoke("is suicide covered in the policy?")  

{'query': 'is suicide covered in the policy?',
 'result': 'No, suicide is not covered in the policy. According to the context, India. d Attempt at suicide is listed as an exclusion, which means that the policy does not provide coverage for suicide attempts.'}

In [ ]:
# ## Cosine Similarity Retreive Results from VectorDB
# def retrieve_query(query,k=2):
#     matching_results=index.similarity_search(query,k=k)
#     return matching_results

In [ ]:
# from langchain.chains.question_answering import load_qa_chain
# from langchain import OpenAI

In [ ]:
# llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
# chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
# ## Search answers from VectorDB
# def retrieve_answers(query):
#     doc_search=retrieve_query(query)
#     print(doc_search)
#     response=chain.run(input_documents=doc_search,question=query)
#     return response

In [ ]:
# our_query = "How much the agriculture target will be increased by how many crore?"
# answer = retrieve_answers(our_query)
# print(answer)